# USG MatReader Class

In [ ]:
class USG_MatReader():

    def __init__(self, file_index, offset_dict=None):
        self.root = '/content/drive/MyDrive/Projects/Thesis/Data/'
        self.file_names = [fn for fn in os.listdir(self.root) if fn.startswith('usData')]
        print(self.file_names)

        self.chosen_file_name = self.file_names[file_index]

        f = h5py.File( self.root+self.chosen_file_name, 'r' )
        self.images = np.array( f['usdata']['avi_crop'] )
        self.coordinates = np.array( f['usdata']['coordinates'] )[0]
        self.depth = f['usdata']['depth']
        self.refs = f['#refs#']
        self.masked_ids = self.get_masked_ids()
        
        if offset_dict == None:
            self.offset_dict = {}
        else:
            self.offset_dict = offset_dict

    def __len__( self ):
        return self.images.shape[0]

    def __getitem__(self, i):
        return self.images[i].T

    # Searches for masked frames
    def get_masked_ids( self ):
        masked_img_ids = []
        for img_id in range(len(self)):
            ref = self.coordinates[img_id]
            mask = np.array(self.refs[ref])
            if mask.ndim == 2:
                masked_img_ids.append( img_id )
        n = len(masked_img_ids)
        print( f'Images with mask: {n}/{len(self)} ({round(100*n/len(self), 2)}%)' )
        return masked_img_ids

    # Stick a mask to the image
    def get_ground_truth( self, img_id:int ) -> np.array:
        
        img = self.images[img_id]
        if not (img_id in self.masked_ids):
            print(f'Image {img_id} doesn\'t have a mask.')
            return img

        mask = self.get_mask(img_id).T
        over = overlap(img, mask)
        # over = np.transpose(over, (2, 1, 0))
        return over

    # Fetch a 2D mask for a particular frame
    def get_mask( self, img_id:int ) -> np.array:
        
        img = self.images[img_id]
        canvas2D = np.zeros(img.shape)

        if not (img_id in self.masked_ids):
            print(f'Image {img_id} doesn\'t have a mask.')
            return canvas2D

        ref = self.coordinates[ img_id ]
        mask = np.array(self.refs[ ref ])

        # Check if the values in the mask exceed
        # the image dimensions (it happens)
        xmax = np.max( mask[:,0] )
        ymax = np.max( mask[:,1] )
        x_offset = int(xmax - img.shape[0])
        x_offset = 0 if (x_offset < 0) else x_offset+1
        y_offset = int(ymax - img.shape[1])
        y_offset = 0 if (y_offset < 0) else y_offset+1

        offset = self.offset_dict.get( self.chosen_file_name )

        for point in mask:
            x = int(point[0])-x_offset 
            y = int(point[1])-y_offset
            if offset:
                x += offset
            if x>=0:
                canvas2D[x][y] = 1

        return canvas2D.T


    """ Returns a Ground Truth as yx double-vector. """
    def get_encoded_mask( self, frame_id ):
        ref = self.coordinates[ frame_id ]
        mask = np.array(self.refs[ ref ])
        mask = np.fliplr(mask)
        return mask


# Show Functions

In [ ]:
def show(  img:np.array,  title:str="",  new:bool=True  ) -> None:
    with plt.style.context('dark_background'):
        if new:
            plt.figure(figsize=(14, 10), dpi=120)
        plt.title( title )
        if img.ndim == 2:
            # img = np.transpose(img, (1, 0))
            plt.imshow( img, cmap='gray')
        else:
            img = np.transpose(img, (1, 0, 2))
            plt.imshow( img, cmap='viridis')


In [ ]:
def shows(
     imgs:       List[np.array],
     ncols:      int = 3,
     dpi:        int = 80,
     titles:     List[str] = None
    ) -> None:

    with plt.style.context('dark_background'):
        IMG_SIZE = 10
        N = len(imgs)
        if not titles:
            titles = [' '] * N

        nrows = np.ceil( N / ncols ).astype(int)
        figsize_col = IMG_SIZE * ncols
        figsize_row = IMG_SIZE * nrows

        plt.figure( figsize=(figsize_col//2, figsize_row), dpi=dpi )
        for i in range(N):
            plt.subplot( nrows, ncols, i+1 )
            # img = np.transpose(imgs[i], (0, 2, 1))
            show( imgs[i], titles[i], new=False )


In [ ]:
"""Place the colored mask over the image."""
def overlap( img: np.array, mask: np.array, color=None ) -> None:

    if color == None:
        # color = (0, 0, 255) # monoblue
        color = (124, 252, 0) # lawngreen
        # color = (48, 213, 200) # turquoise
        # color = (255, 0, 0) # mono red
    canvas3D = np.zeros( (*img.shape, 3) )
    for i in range(3):
        layer = np.zeros( img.shape )
        layer += img[:,:]
        mask_layer = mask[:,:] * (color[i] / 255)
        np.copyto( layer, mask_layer, where=(mask_layer != 0) )
        canvas3D[:,:,i] = layer
    canvas3D = np.transpose(canvas3D, (2, 0, 1))
    return canvas3D.T